# Analysis prompt 1

"What factors best predict whether an individual is categorized as in permanent housing or not in permanent housing"?


## Purpose

The purpose of this notebook is to give an indication about which factors contribute to a succesful leave out of homelessness. 

### Requirements

A DataFrame will be created that contains several attributes about an individual. 
This means that the DataFrame is aggregated at the individual level.

- Author: Annalie Kruseman
- Date: 2016-10-31

In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
# %load_ext autoreload
# # the "1" means: always reload modules marked with "%aimport"
# %autoreload 1

from __future__ import absolute_import, division, print_function
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os, sys
# from tqdm import tqdm
# import warnings

sns.set_context("poster", font_scale=1.0)

In [2]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [3]:
# where the data is stored

matt = 'SampleDataSet-MOSBE&SCz-2012-1001--2016-0531/USE THIS ONE/'
annalie = 'data/c4sf/DATA_CTA/'

datadir = os.path.join(os.getenv('HOME'), annalie)

In [4]:
def encode_boolean(df, col):
    '''Encode values as booleans.
    If the string is 'Yes', the new value will be True. Otherwise it will be False.
    '''
    df.loc[df[col] == 'Yes', col] = True
    df.loc[df[col] == 'No', col] = False
    df.loc[df[col] == 'Not Applicable - Child', col] = False
    df.loc[df[col].isin(['Client refused',
                         "Client doesn't know",
                         'Data not collected',
                         '',
                         np.nan]), col] = False
    return df

In [5]:
def encode_unknown(df, col):
    '''Change non-informative values to 'Unknown'.
    '''
    df.loc[df[col].isin(['Client refused',
                         'Refused',
                         "Client doesn't know",
                         'Data not collected']), col] = 'Unknown'
    return df

In [6]:
sheet = 'Client'

cols = [
    'Personal ID',
    'Race',
    'Ethnicity',
    'Gender',
    'Veteran Status',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_client = pd.read_csv(infile, header=0, index_col=0, usecols=cols, sep=';')
df_client = df_client.dropna(how='all')
df_client.index = df_client.index.astype('int')

# drop people that we don't have demographic information for
df_client = df_client.dropna(how='any', subset=['Race', 'Ethnicity', 'Gender'])

# fill in missing values
df_client['Veteran Status'] = df_client['Veteran Status'].fillna(value='')

# Remove "(HUD) from strings
cols = ['Race', 'Ethnicity', 'Veteran Status']
for col in cols:
    df_client[col] = df_client[col].apply(lambda x: x.replace(' (HUD)', ''))

# put the nans back
df_client.loc[df_client['Veteran Status'] == '', 'Veteran Status'] = np.nan

# and encode booleans
df_client = encode_boolean(df_client, 'Veteran Status')

# Some are unknown
df_client = encode_unknown(df_client, 'Race')
df_client = encode_unknown(df_client, 'Ethnicity')
df_client = encode_unknown(df_client, 'Gender')

In [7]:
sheet = 'Enrollment'

cols = [
    'Personal ID',
    'Project Entry ID',
    'Client Age at Entry',
    'Last Permanent Zip',
    'Entry Date',
    'Exit Date',
    'Project ID',
    'Housing Status @ Project Start',
    'Living situation before program entry?',
    'Client Location',
    'Household ID',
    'Relationship to HoH',
    'Disabling Condition',
    'Continuously Homeless One Year',
    'Times Homeless Past Three Years',
    'Months Homeless This Time',
    'Chronic Homeless',
    'In Permanent Housing',
    'Residential Move In Date',
    'Domestic Violence Victim',
    'DV When Occurred',
    'DV Currently Fleeing',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_enroll = pd.read_csv(infile, header=0, index_col=0, usecols=cols, sep=';',
                        parse_dates=['Entry Date', 'Exit Date', 'Residential Move In Date'],
                        infer_datetime_format=True)

df_enroll = df_enroll.dropna(axis=0, how='all')
df_enroll.index = df_enroll.index.astype('int')

# drop anyone for whom we don't have age
df_enroll = df_enroll.dropna(subset=['Client Age at Entry'])

# turn these into integers
cols = ['Project Entry ID', 'Client Age at Entry', 'Project ID', 'Household ID']
for col in cols:
    df_enroll[col] = df_enroll[col].astype('int')

# Remove "(HUD) from strings
cols = ['Housing Status @ Project Start',
        'Living situation before program entry?',
        'Disabling Condition',
        'Continuously Homeless One Year',
        'Domestic Violence Victim',
        'DV When Occurred',
        'DV Currently Fleeing',
        ]
for col in cols:
    df_enroll[col] = df_enroll[col].fillna(value='')
    df_enroll[col] = df_enroll[col].apply(lambda x: x.replace(' (HUD)', ''))
    # put the nans back
    df_enroll.loc[df_enroll[col] == '', col] = np.nan

# encode booleans

# cols = [
#     'Disabling Condition',
#     'Continuously Homeless One Year',
#     'Chronic Homeless',
#     'In Permanent Housing',
#     'Domestic Violence Victim',
#     'DV Currently Fleeing', ]

col = 'Disabling Condition'
df_enroll = encode_boolean(df_enroll, col)

col = 'Continuously Homeless One Year'
df_enroll = encode_boolean(df_enroll, col)

col = 'Chronic Homeless'
df_enroll = encode_boolean(df_enroll, col)

col = 'In Permanent Housing'
df_enroll = encode_boolean(df_enroll, col)

col = 'Domestic Violence Victim'
df_enroll = encode_boolean(df_enroll, col)

col = 'DV Currently Fleeing'
df_enroll = encode_boolean(df_enroll, col)

In [8]:
# one person has a negative age. make it positive.
col = 'Client Age at Entry'
df_enroll.loc[df_enroll[col] < 0, col] = df_enroll.loc[df_enroll[col] < 0, col] * -1

In [9]:
sheet = 'Disability'

cols = [
    'Personal ID',
    'Disability Type',
    'Receiving Services For',
    'Disabilities ID',
    'Project Entry ID',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_disability = pd.read_csv(infile, header=0, index_col=0, usecols=cols, sep=',')
df_disability = df_disability.dropna(axis=0, how='all')
# df_disability.index = df_disability.index.astype('int') # TypeError: Setting dtype to anything other than float64 or object is not supported

# turn these into integers
cols = ['Disabilities ID', 'Project Entry ID']
for col in cols:
    df_disability[col] = df_disability[col].astype('int')

# Remove "(HUD) from strings
cols = ['Disability Type',
        'Receiving Services For',
        ]
for col in cols:
    df_disability[col] = df_disability[col].fillna(value='')
    df_disability[col] = df_disability[col].apply(lambda x: x.replace(' (HUD)', ''))
    # put the nans back
    df_disability.loc[df_disability[col] == '', col] = np.nan

# encode booleans
col = 'Receiving Services For'
df_disability = encode_boolean(df_disability, col)

In [10]:
sheet = 'HealthInsurance'

cols = [
    'Personal ID',
    'Health Insurance Information Date',
    'Health Insurance',
    'Data Collection Stage',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_healthins = pd.read_csv(infile, header=0, index_col=0, usecols=cols,
                           parse_dates=['Health Insurance Information Date'],
                           infer_datetime_format=True)

df_healthins = df_healthins.dropna(axis=0, how='all')
# df_healthins.index = df_healthins.index.astype('int')

In [11]:
sheet = 'Benefit'

cols = [
    'Personal ID',
    'Non-Cash Benefit',
    'Data Collection Stage',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_benefit = pd.read_csv(infile, header=0, index_col=0, usecols=cols, sep=';')

df_benefit = df_benefit.dropna(axis=0, how='all')
df_benefit.index = df_benefit.index.astype('int')

# Drop any project missing the code
df_benefit = df_benefit.dropna(how='any', subset=['Non-Cash Benefit'])

# Remove "(HUD) from strings
cols = ['Non-Cash Benefit',
        ]
for col in cols:
    df_benefit[col] = df_benefit[col].fillna(value='')
    df_benefit[col] = df_benefit[col].apply(lambda x: x.replace(' (HUD)', ''))
    # put the nans back
    df_benefit.loc[df_benefit[col] == '', col] = np.nan

# shorten some column values
col = 'Non-Cash Benefit'
df_benefit.loc[df_benefit[col] == 'Supplemental Nutrition Assistance Program (Food Stamps)', col] = 'Food Stamps'
df_benefit.loc[df_benefit[col] == 'Special Supplemental Nutrition Program for WIC', col] = 'WIC'
df_benefit.loc[df_benefit[col] == 'Section 8, Public Housing, or other ongoing rental assistance', col] = 'Section 8, Public Housing'

In [12]:
sheet = 'Income Entry & Exit'

cols = [
    'Personal ID',
    'Project Entry ID',
    'Entry Alimony',
    'Entry Child Support',
    'Entry Earned',
    'Entry GA',
    'Entry Other',
    'Entry Pension',
    'Entry Private Disability',
    'Entry Social Security Retirement',
    'Entry SSDI',
    'Entry SSI',
    'Entry TANF',
    'Entry Unemployment',
    'Entry VA Non-Service',
    'Entry VA Service Connected',
    "Entry Worker's Compensation",
    'Entry Total Income',
    'Exit Alimony',
    'Exit Child Support',
    'Exit Earned',
    'Exit GA',
    'Exit Other',
    'Exit Pension',
    'Exit Private Disability',
    'Exit Social Security Retirement',
    'Exit SSDI',
    'Exit SSI',
    'Exit TANF',
    'Exit Unemployment',
    'Exit VA Non-Service',
    'Exit VA Service Connected',
    "Exit Worker's Compensation",
    'Exit Total Income',
    'Income Change',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_income = pd.read_csv(infile, header=0, index_col=0, usecols=cols)

df_income = df_income.dropna(axis=0, how='all')
# df_income.index = df_income.index.astype('int')

# turn these into integers
cols = ['Project Entry ID']
for col in cols:
    df_income[col] = df_income[col].astype('int')

# assume all nans are $0
df_income = df_income.fillna(value='0')

# turn the dollar strings into integers
for col in df_income.columns:
    if col != 'Project Entry ID':
        df_income[col] = df_income[col].str.replace(',', '')
        df_income[col] = df_income[col].str.replace(r'[^-+\d.]', '').astype(int)

In [13]:
sheet = 'Service'

cols =  [
    'Personal ID',
    'Services ID',
    'Date Provided',
    'Date Ended',
    'Service Code',
    'Description',
    'Project ID',
    'Record Type',
    'Project Entry ID',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_service = pd.read_csv(infile, header=0, index_col=0, usecols=cols, sep=';',
                         parse_dates=['Date Provided', 'Date Ended'],
                         infer_datetime_format=True)

df_service = df_service.dropna(axis=0, how='all')
df_service.index = df_service.index.astype('int')

# Drop anyone missing these IDs
df_service = df_service.dropna(how='any', subset=['Project ID', 'Project Entry ID'])

# turn these into integers
cols = ['Project ID', 'Project Entry ID']
for col in cols:
    df_service[col] = df_service[col].astype('int')

In [14]:
# drop any null dates
df_service = df_service.dropna(subset=['Date Provided', 'Date Ended'])

In [15]:
sheet = 'Project'

cols = [
    'Project ID',
    'Project Name',
    'Project Type Code',
    'Address City',
    'Organization Name',
    'Address Postal Code'
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_project = pd.read_csv(infile, header=0, index_col=1, usecols=cols, sep=';')

df_project.head()

df_project = df_project.dropna(axis=0, how='all')
df_project.index = df_project.index.astype('int')

# Drop any project missing the code
df_project = df_project.dropna(how='any', subset=['Project Type Code'])

# Remove "(HUD) from strings
cols = ['Project Type Code',
        ]
for col in cols:
    df_project[col] = df_project[col].fillna(value='')
    df_project[col] = df_project[col].apply(lambda x: x.replace(' (HUD)', ''))
    # put the nans back
    df_project.loc[df_project[col] == '', col] = np.nan

In [16]:
sheet = 'BedInventory'

cols = [
    'Project ID',
    'Inventory ID',
    'Inventory Household Type',
    'HMIS Participating Beds',
    'Inventory Start Date',
    'Inventory End Date',
    'Unit Inventory',
    'Bed Inventory',
    'Vet Bed Inventory',
    'Youth Bed Inventory',
    'Youth Bed Age Group',
    ]

infile = os.path.join(datadir, '{s}.csv'.format(s=sheet))

df_bedinv = pd.read_csv(infile, header=0, index_col=0, usecols=cols,
                        parse_dates=['Inventory Start Date', 'Inventory End Date'],
                        infer_datetime_format=True)

df_bedinv = df_bedinv.dropna(axis=0, how='all')
# df_bedinv.index = df_bedinv.index.astype('int')

# turn these into integers, assume zero if NaN
cols = ['Inventory ID', 'HMIS Participating Beds', 'Unit Inventory', 'Bed Inventory', 'Vet Bed Inventory', 'Youth Bed Inventory']
for col in cols:
    # df_bedinv[col] = df_bedinv.loc[~df_bedinv[col].isnull(), col].apply(lambda x: int(x))
    df_bedinv[col] = df_bedinv[col].fillna(value=0)
    df_bedinv[col] = df_bedinv[col].astype('int')

# Clean data and make some features

In [17]:
# Only keep rows with entry dates starting in 2011
df_enroll = df_enroll[df_enroll['Entry Date'] >= '2011']

# Only keep rows with exit date
df_enroll = df_enroll[df_enroll['Exit Date'].notnull()]

In [18]:
# calculate the number of days that someone was enrolled
df_enroll['Days Enrolled'] = ((df_enroll['Exit Date'] - df_enroll['Entry Date']) / np.timedelta64(1, 'D')).astype(int)

In [19]:
# remove anyone with negative number of enrollment days
df_enroll = df_enroll[df_enroll['Days Enrolled'] >= 0]

In [20]:
# Join the client information with enrollment information.
# Inner join because we need both.
df = df_client.merge(df_enroll, how='inner', left_index=True, right_index=True)

In [21]:
# Join the merged new dataframe with benefit information.
df = df.merge(df_benefit.reset_index().groupby(by=['Personal ID'])[['Non-Cash Benefit']].nth(0),
              how='left', left_index=True, right_index=True)

df['Non-Cash Benefit'] = df['Non-Cash Benefit'].fillna('None')

In [22]:
# Join the merged new dataframe with disability information.
df = df.merge(df_disability.reset_index().groupby(by=['Personal ID'])[['Disability Type']].nth(0),
              how='left', left_index=True, right_index=True)

df['Disability Type'] = df['Disability Type'].fillna('None')

In [23]:
# merge organizations from the project dataframe with the merged client and enrollment dataframe

# make column of index from df dataframe (Personal ID)
df_resetindex = df.reset_index()
# make column of index from project dataframe (Project ID)
df_project_resetindex = df_project.reset_index()

# merge new df dataframe with project dataframe on Project ID
df_dfProject = df_resetindex.merge(df_project_resetindex, how='left', on='Project ID').set_index('Personal ID').sort_index()

In [24]:
# when 'Days Enrolled' is 0 and 'Non-Cash Benefit' is foodstamp, rename 0 value to 1
df_dfProject.ix[(df_dfProject['Days Enrolled'] == 0) & (df_dfProject['Non-Cash Benefit'] == 'Food Stamps'), 'Days Enrolled'] = 1

In [25]:
df_dfProject['Non-Cash Benefit'].value_counts()

None                            39890
Food Stamps                     20938
Other Source                      992
Section 8, Public Housing         964
WIC                               611
Other TANF-Funded Services        183
TANF Transportation Services       42
TANF Child Care Services           33
Temporary rental assistance        13
Name: Non-Cash Benefit, dtype: int64

    NB: What does it mean that someone doesn't receive a 'Non-Cash Benefit' (value = None), but for example, is
    registered for Emergency Shelter?

In [26]:
# remove all 'None' values for 'Non-Cash Benefit'
df_dfProject = df_dfProject[df_dfProject['Non-Cash Benefit'] != 'None']

In [27]:
df_dfProject['Non-Cash Benefit'].value_counts()

Food Stamps                     20938
Other Source                      992
Section 8, Public Housing         964
WIC                               611
Other TANF-Funded Services        183
TANF Transportation Services       42
TANF Child Care Services           33
Temporary rental assistance        13
Name: Non-Cash Benefit, dtype: int64

In [28]:
df = df_dfProject

In [29]:
# sort by entry date
df = df.sort('Entry Date')

/Users/annalie/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [30]:
# set up to count the number of times a person was in the system
df['Enrollments'] = 1

In [31]:
agg = {'In Permanent Housing': 'last',
       'Enrollments': 'sum',
       'Race': 'first',
       'Ethnicity': 'first',
       'Gender': 'first',
       'Veteran Status': 'max',
       'Client Age at Entry': 'last',
       'Days Enrolled': 'sum',
       'Domestic Violence Victim': 'max',
       'Disability Type': 'last',
       'Non-Cash Benefit': 'last',
       'Housing Status @ Project Start': 'last',
       'Living situation before program entry?': 'last',
       'Continuously Homeless One Year': 'max',
#        'Months Homeless This Time': 'last',
       'Chronic Homeless': 'max',
       'Project Name': 'last',
       'Organization Name': 'last',
       'Project Type Code': 'last',
#        'Address City': 'last',
#        'Address Postal Code': 'last',
      }
df_features = df.reset_index().groupby(by=['Personal ID']).agg(agg)

In [32]:
# replace values in the below columns to values where False = 0 and True = 1
df_features['In Permanent Housing'] = df_features['In Permanent Housing'].map({False: 0, True: 1})
df_features['Chronic Homeless'] = df_features['Chronic Homeless'].map({False: 0, True: 1})
df_features['Domestic Violence Victim'] = df_features['Domestic Violence Victim'].map({False: 0, True: 1})
df_features['Veteran Status'] = df_features['Veteran Status'].map({False: 0, True: 1})
df_features['Continuously Homeless One Year'] = df_features['Continuously Homeless One Year'].map({False: 0, True: 1})

    Remove values with missing values in the column "Months Homeless This Time"

In [33]:
# number of missing values in column 'Months Homeless This Time":
# df_features['Months Homeless This Time'].isnull().sum().sum()

In [34]:
# number of total values
len(df_features)

3397

In [35]:
df_features.head(1)

,Non-Cash Benefit,Chronic Homeless,Domestic Violence Victim,Housing Status @ Project Start,Gender,Veteran Status,Days Enrolled,Disability Type,Living situation before program entry?,In Permanent Housing,Race,Client Age at Entry,Project Name,Enrollments,Organization Name,Project Type Code,Ethnicity,Continuously Homeless One Year
Personal ID,,,,,,,,,,,,,,,,,,
173781,Food Stamps,0,1,Category 1 - Homeless,Female,0,148,None,"Emergency shelter, including hotel or motel pa...",0,White,35,MOSBE SOP - Natividad Shelter,2,MOSBE SOP,Emergency Shelter,Hispanic/Latino,0


# Build a Logistic Regression model
    (1) Scikit Learn, see below
    (2) Statsmodels, see bottom

# (1) Logistic Regression with Scikit Learn

In [36]:
# create dummy variables for the following variables
for column in ['Non-Cash Benefit', 
               'Housing Status @ Project Start', 
               'Gender', 
               'Disability Type', 
               'Living situation before program entry?', 
               'Race', 
               'Project Name', 
               'Organization Name', 
               'Project Type Code', 
               'Ethnicity']:
    dummies = pd.get_dummies(df_features[column])
    df_features[dummies.columns] = dummies
    
# create a new dataframe that contains only numeric datatypes
df_features_numeric = df_features.drop(['Non-Cash Benefit', 'Housing Status @ Project Start', 'Domestic Violence Victim', 'Gender', 'Disability Type',
                                        'Living situation before program entry?','Race', 'Project Name', 'Organization Name',
                                        'Project Type Code', 'Ethnicity'], axis=1)

df_features_numeric.head(1)

,Chronic Homeless,Veteran Status,Days Enrolled,In Permanent Housing,Client Age at Entry,Enrollments,Continuously Homeless One Year,Food Stamps,Other Source,Other TANF-Funded Services,"Section 8, Public Housing",TANF Child Care Services,TANF Transportation Services,Temporary rental assistance,WIC,At-risk of homelessness,Category 1 - Homeless,Category 2 - At imminent risk of losing housing,Category 3 - Homeless only under other federal statutes,Category 4 - Fleeing domestic violence,Client doesn't know,Client refused,Data not collected,Stably housed,Female,Male,Other,Transgender female to male,Transgender male to female,Unknown,Alcohol Abuse,Both Alcohol and Drug Abuse,Chronic Health Condition,Developmental,Drug Abuse,HIV/AIDS,Mental Health Problem,None,Physical,Substance Abuse,"Emergency shelter, including hotel or motel paid for with emergency shelter voucher(HUD)",Foster care home or foster care group home,Hospital or other residential non-psychiatric medical facility,Hotel or motel paid for without emergency shelter voucher,"Jail, prison or juvenile detention facility","Owned by client, no ongoing housing subsidy","Owned by client, with ongoing housing subsidy",Permanent housing for formerly homeless persons,Place not meant for habitation,Psychiatric hospital or other psychiatric facility,"Rental by client, no ongoing housing subsidy","Rental by client, with VASH subsidy","Rental by client, with other ongoing housing subsidy",Residential project or halfway house with no homeless criteria,Safe Haven,"Staying or living in a family member's room, apartment or house","Staying or living in a friend's room, apartment or house",Substance abuse treatment facility or detox center,Transitional housing for homeless persons (including homeless youth),American Indian or Alaska Native,Asian,Black or African American,Native Hawaiian or Other Pacific Islander,White,MOSBE SOP - Transitional Housing - Men In Transition,MOSBE - Homeless Winter Shelter for Families,MOSBE CHS - Elm House,MOSBE CHS - RHY - BCP - HP,MOSBE CHS - RHY SOP,MOSBE CHS - Safe Passage,MOSBE CHS - Safe Place Warming Shelter,MOSBE CSUMB - Chinatown Project,MOSBE Franciscan Workers - House of Peace,MOSBE Franciscan Workers - Women Alive! Shelter,MOSBE Franciscan Workers of Junipero Serra,MOSBE HRC - LEGACY Emergency Rental Assistance,MOSBE HRC - SSVF- P1 - CCCIL HP,MOSBE HRC - SSVF- P1 - CCCIL RRH,MOSBE HRC - SSVF- P1 - HRC HP,MOSBE HRC - SSVF- P1 - HRC RRH,MOSBE HRC - SSVF- P1 - VTC HP,MOSBE HRC - SSVF- P1 - VTC RRH,MOSBE HRC - SSVF- P2 HRC- RRH - Santa Cruz Clients,MOSBE HRC - SSVF- P2 - HRC RRH,MOSBE HRC - SSVF- P2 HRC - HP- Monterey/San Benito Clients,MOSBE HRC - SSVF- P2 HRC - RRH- Monterey/San Benito Clients,MOSBE HRC - Security Deposit Guarantees,MOSBE Housing Authority - Pueblo del Mar Family Recovery Community,MOSBE Housing Authority - S + C Vouchers,MOSBE Housing Resource Center (HRC)*,MOSBE Interim,MOSBE Interim - MCHOME Enrolled,MOSBE Interim - MCHOME Outreach,MOSBE Interim - MCHOME S+C,MOSBE Interim - MCHOPE,MOSBE Interim - Sandy Shores,MOSBE Interim - Shelter Cove,MOSBE Interim - Sunflower Gardens - PSH,MOSBE Interim - Sunflower Gardens - THU,MOSBE SOP - Hamilton ES,MOSBE SOP - Hamilton Family Units -DO NOT USE-,MOSBE SOP - MLP - Emergency Men's Program,MOSBE SOP - Mobile Outreach Shelter Program - MOST,MOSBE SOP - Natividad Shelter,MOSBE SOP - Transitional Housing - Lexington Court,MOSBE SOP - Transitional Housing - Wittenmyer (Homeward Bound),MOSBE SOP - Winter Warming Shelter,MOSBE San Benito County - Helping Hands,MOSBE San Benito County - Homeless Warming Shelter,MOSBE The Salvation Army - Casa de las Palmas,MOSBE The Salvation Army - Fredericksen House,MOSBE The Salvation Army - Good Samaritan Center,MOSBE The Salvation Army - Phase II,MOSBE VTC DO NOT ENTER HERE,MOSBE Veterans Transition Center - GPD Housing,MOSBE Veterans Transition Center - Outreach,SCz - CAB TSP Rental Assistance,SCz - County Office of Education,SCz - Encompass,SCz - Encompass - Ander

In [73]:
df_features_numeric.shape

(3397, 199)

# Create dataframes with dummie variables for the variables seperately

In [37]:
## NOT NECCESSARY FOR STATSMODELS

# create dataframe for only 'Non Cash Benefit' with dummy variables
df_features_non_cash_benefit = df_features[['In Permanent Housing', 'Non-Cash Benefit']]

# create dummy variables for the following variables
dummies_non_cash_benefit = pd.get_dummies(df_features_non_cash_benefit['Non-Cash Benefit'])
df_features_non_cash_benefit[dummies_non_cash_benefit.columns] = dummies_non_cash_benefit

# create a new dataframe that contains only numeric datatypes
df_features_non_cash_benefit = df_features_non_cash_benefit.drop('Non-Cash Benefit', axis=1)
df_features_non_cash_benefit.head()

/Users/annalie/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:2440: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,In Permanent Housing,Food Stamps,Other Source,Other TANF-Funded Services,"Section 8, Public Housing",TANF Child Care Services,TANF Transportation Services,Temporary rental assistance,WIC
Personal ID,,,,,,,,,
173781,0,1,0,0,0,0,0,0,0
173803,0,1,0,0,0,0,0,0,0
173848,0,1,0,0,0,0,0,0,0
173885,0,1,0,0,0,0,0,0,0
173899,0,1,0,0,0,0,0,0,0


In [38]:
## NOT NECCESSARY FOR STATSMODELS

# create dataframe for only 'Housing Status @ Project Start' with dummy variables
df_features_housing_status = df_features[['In Permanent Housing', 'Housing Status @ Project Start']]

# create dummy variables for the following variables
dummies_housing_status = pd.get_dummies(df_features_housing_status['Housing Status @ Project Start'])
df_features_housing_status[dummies_housing_status.columns] = dummies_housing_status

# create a new dataframe that contains only numeric datatypes
df_features_housing_status = df_features_housing_status.drop('Housing Status @ Project Start', axis=1)
df_features_housing_status.head()

,In Permanent Housing,At-risk of homelessness,Category 1 - Homeless,Category 2 - At imminent risk of losing housing,Category 3 - Homeless only under other federal statutes,Category 4 - Fleeing domestic violence,Client doesn't know,Client refused,Data not collected,Stably housed
Personal ID,,,,,,,,,,
173781,0,0,1,0,0,0,0,0,0,0
173803,0,0,1,0,0,0,0,0,0,0
173848,0,0,1,0,0,0,0,0,0,0
173885,0,0,1,0,0,0,0,0,0,0
173899,0,0,1,0,0,0,0,0,0,0


In [39]:
## NOT NECCESSARY FOR STATSMODELS

# create dataframe for only 'Living situation before program entry?' with dummy variables
df_features_living_situation = df_features[['In Permanent Housing', 'Living situation before program entry?']]

# create dummy variables for the following variables
dummies_living_situation = pd.get_dummies(df_features_living_situation['Living situation before program entry?'])
df_features_living_situation[dummies_living_situation.columns] = dummies_living_situation

# create a new dataframe that contains only numeric datatypes
df_features_living_situation = df_features_living_situation.drop('Living situation before program entry?', axis=1)
df_features_living_situation.head()

,In Permanent Housing,Client doesn't know,Client refused,Data not collected,"Emergency shelter, including hotel or motel paid for with emergency shelter voucher(HUD)",Foster care home or foster care group home,Hospital or other residential non-psychiatric medical facility,Hotel or motel paid for without emergency shelter voucher,"Jail, prison or juvenile detention facility",Other,"Owned by client, no ongoing housing subsidy","Owned by client, with ongoing housing subsidy",Permanent housing for formerly homeless persons,Place not meant for habitation,Psychiatric hospital or other psychiatric facility,"Rental by client, no ongoing housing subsidy","Rental by client, with VASH subsidy","Rental by client, with other ongoing housing subsidy",Residential project or halfway house with no homeless criteria,Safe Haven,"Staying or living in a family member's room, apartment or house","Staying or living in a friend's room, apartment or house",Substance abuse treatment facility or detox center,Transitional housing for homeless persons (including homeless youth)
Personal ID,,,,,,,,,,,,,,,,,,,,,,,,
173781,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
173803,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
173848,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
173885,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
173899,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
## NOT NECCESSARY FOR STATSMODELS

# create dataframe for only 'Project Name' with dummy variables
df_features_project_name = df_features[['In Permanent Housing', 'Project Name']]

# create dummy variables for the following variables
dummies_project_name = pd.get_dummies(df_features_project_name['Project Name'])
df_features_project_name[dummies_project_name.columns] = dummies_project_name

# create a new dataframe that contains only numeric datatypes
df_features_project_name = df_features_project_name.drop('Project Name', axis=1)
df_features_project_name.head()

,In Permanent Housing,MOSBE SOP - Transitional Housing - Men In Transition,MOSBE - Homeless Winter Shelter for Families,MOSBE CHS - Elm House,MOSBE CHS - RHY - BCP - HP,MOSBE CHS - RHY SOP,MOSBE CHS - Safe Passage,MOSBE CHS - Safe Place Warming Shelter,MOSBE CSUMB - Chinatown Project,MOSBE Franciscan Workers - House of Peace,MOSBE Franciscan Workers - Women Alive! Shelter,MOSBE Franciscan Workers of Junipero Serra,MOSBE HRC - LEGACY Emergency Rental Assistance,MOSBE HRC - SSVF- P1 - CCCIL HP,MOSBE HRC - SSVF- P1 - CCCIL RRH,MOSBE HRC - SSVF- P1 - HRC HP,MOSBE HRC - SSVF- P1 - HRC RRH,MOSBE HRC - SSVF- P1 - VTC HP,MOSBE HRC - SSVF- P1 - VTC RRH,MOSBE HRC - SSVF- P2 HRC- RRH - Santa Cruz Clients,MOSBE HRC - SSVF- P2 - HRC RRH,MOSBE HRC - SSVF- P2 HRC - HP- Monterey/San Benito Clients,MOSBE HRC - SSVF- P2 HRC - RRH- Monterey/San Benito Clients,MOSBE HRC - Security Deposit Guarantees,MOSBE Housing Authority - Pueblo del Mar Family Recovery Community,MOSBE Housing Authority - S + C Vouchers,MOSBE Housing Resource Center (HRC)*,MOSBE Interim,MOSBE Interim - MCHOME Enrolled,MOSBE Interim - MCHOME Outreach,MOSBE Interim - MCHOME S+C,MOSBE Interim - MCHOPE,MOSBE Interim - Sandy Shores,MOSBE Interim - Shelter Cove,MOSBE Interim - Sunflower Gardens - PSH,MOSBE Interim - Sunflower Gardens - THU,MOSBE SOP - Hamilton ES,MOSBE SOP - Hamilton Family Units -DO NOT USE-,MOSBE SOP - MLP - Emergency Men's Program,MOSBE SOP - Mobile Outreach Shelter Program - MOST,MOSBE SOP - Natividad Shelter,MOSBE SOP - Transitional Housing - Lexington Court,MOSBE SOP - Transitional Housing - Wittenmyer (Homeward Bound),MOSBE SOP - Winter Warming Shelter,MOSBE San Benito County - Helping Hands,MOSBE San Benito County - Homeless Warming Shelter,MOSBE The Salvation Army - Casa de las Palmas,MOSBE The Salvation Army - Fredericksen House,MOSBE The Salvation Army - Good Samaritan Center,MOSBE The Salvation Army - Phase II,MOSBE VTC DO NOT ENTER HERE,MOSBE Veterans Transition Center - GPD Housing,MOSBE Veterans Transition Center - Outreach,SCz - CAB TSP Rental Assistance,SCz - County Office of Education,SCz - Encompass,SCz - Encompass - Anderson,SCz - Encompass - River Street Shelter,SCz - Encompass - River Street Shelter PATH,SCz - FIT - Brommer,SCz - FIT - HP SSVF - FIT Clients,SCz - FIT - HP SSVF - HSC Clients,SCz - FIT - HP SSVF - PVSS Clients,SCz - FIT - HUD (Scattered Site),SCz - FIT - HUD/DA (Clean & Sober),SCz - FIT - RRH SSVF - FIT Clients,SCz - FIT - RRH SSVF - HSC Clients,SCz - FIT - RRH SSVF - PVSS Clients,SCz - Front St.,SCz - Front St. - Paget Center,SCz - HPHP-MATCH Housing Inebriates I,SCz - HPHP-MATCH Housing Inebriates III,SCz - HSC - 180/2020,SCz - HSC - NO DATA HERE,SCz - HSC - Page Smith Community House,SCz - HSC - Paul Lee Loft,SCz - HSC - Rebele Family Shelter Program,SCz - HSC - Recuperative Care Center,SCz - HSC - Winter Shelter,SCz - HSD -CHAMP,SCz - HSD -CHAMP- FIT Clients,SCz - PRM - Emergency Shelter,SCz - PVSS - Emergency Shelter,SCz - PVSS - Transitional Housing for Families,SCz - Salvation Army- Winter (Cold Weather) Shelter,SCz-FIT-County Planning Department Grant,SCz-Veterans Resource Center - HP SSVF - Monterey/San Benito Clients,SCz-Veterans Resource Center - RRH SSVF - Monterey/San Benito Clients,SCz-Veterans Resource Center- HP SSVF -San Mateo Clients,SCz-Veterans Resource Center- HP SSVF -Santa Cruz Clients,SCz-Veterans Resource Center- RRH SSVF -San Mateo Clients,SCz-Veterans Resource Center- RRH SSVF -Santa Cruz Clients,SCz-Veterans Resource Center- SSVF - Priority 1 SCz - HP Santa Cruz,SCz-Veterans Resource Center- SSVF - Priority 1 SCz - RRH Santa Cruz,ZZZ* INACTIVE*ZZZ-MOSBE HRC - SSVF-P2 VRSi - Santa Cruz Clients,ZZZ*INACTIVE*ZZZ- MOSBE HRC - SSVF-P2 VRSi- Monterey/San Benito Clients,zzz*Inactive* SCz - CAB - ESG,zzz*Inactive* SCz - FIT - ESG Homeless Prevention,zzz*Inactive* SCz - FIT - ESG Rapid Rehousing,zzz*Inactive* SCz - Salvation Army - Corner House (HUD),zzz*inactive* MOSBE CCHAS (John XXIII) - Voucher Program,zzz*i

In [41]:
## NOT NECCESSARY FOR STATSMODELS

# create dataframe for only 'Project Type Code' with dummy variables
df_features_project_type = df_features[['In Permanent Housing', 'Project Type Code']]

# create dummy variables for the following variables
dummies_project_type = pd.get_dummies(df_features_project_type['Project Type Code'])
df_features_project_type[dummies_project_type.columns] = dummies_project_type

# create a new dataframe that contains only numeric datatypes
df_features_project_type = df_features_project_type.drop('Project Type Code', axis=1)
df_features_project_type.head()

,In Permanent Housing,Emergency Shelter,Homelessness Prevention,Other,PH - Permanent Supportive Housing (disability required for entry),PH - Rapid Re-Housing,Services Only,Street Outreach,Transitional housing
Personal ID,,,,,,,,,
173781,0,1,0,0,0,0,0,0,0
173803,0,1,0,0,0,0,0,0,0
173848,0,0,0,0,0,0,0,0,1
173885,0,0,0,1,0,0,0,0,0
173899,0,1,0,0,0,0,0,0,0


In [72]:
df_features_project_type.shape

(3397, 9)

In [42]:
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

/Users/annalie/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [43]:
# create a feature set and target variable
predictors = df_features_numeric.drop(['In Permanent Housing'], axis=1)
targets = df_features_numeric['In Permanent Housing']

# # Or should I use a list as input?
# predictors_matrix = predictors.as_matrix()
# targets_list = targets.tolist()

In [44]:
# split data set in training and test set
pred_train, pred_test, tar_train, tar_test = train_test_split(predictors, targets, test_size=.3)

# pred_train, pred_test, tar_train, tar_test = train_test_split(predictors_matrix, targets_list, test_size=.3)

In [45]:
# show the shape of the train and test sets
print ("pred_train shape =", pred_train.shape)
print ("pred_test shape =", pred_test.shape)
print ("tar_train shape =", tar_train.shape)
print ("tar_test shape =", tar_test.shape)

pred_train shape = (2377, 198)
pred_test shape = (1020, 198)
tar_train shape = (2377,)
tar_test shape = (1020,)


In [46]:
# fit a logistic regression model to the data
lr_model = LogisticRegression()
lr_model.fit(pred_train, tar_train)
print(lr_model)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [47]:
# make predictions
expected = tar_test
predicted = lr_model.predict(pred_test)

In [48]:
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))
print ("accuracy score:", sklearn.metrics.accuracy_score(expected, predicted))

             precision    recall  f1-score   support

          0       0.95      0.98      0.97       920
          1       0.72      0.57      0.64       100

avg / total       0.93      0.94      0.93      1020

[[898  22]
 [ 43  57]]
accuracy score: 0.936274509804


In [49]:
# plt.scatter(predicted, expected)
# plt.xlabel("predicted probability")
# plt.ylabel("actual outcome")
# plt.title("Logistic REgression Predicted vs. Actual")
# plt.show()

In [50]:
# predict class probabilities of the input samples
predict_probability_lr = lr_model.predict_proba(pred_test)
# list(predict_probability_lr)

In [51]:
# list(predicted)

In [52]:
# get coefficient per predictor

# 1. create list of coefficients and predictors
coef = lr_model.coef_
coef = coef.tolist()
# unnest nested list
coef = sum(coef, [])

predictors_columns = list(predictors.columns)

In [53]:
def mean(x):
    return sum(x) / len(x)

# standard error (deviation from the mean)
def de_mean(x):
    x_bar = mean(x)
    return [x_i - x_bar for x_i in x]

In [54]:
mean(de_mean(predictors['Chronic Homeless']))

2.0007382399172953e-16

In [55]:
st_error = []
for i in pred_test.columns:
    st_error.append(mean(de_mean(predictors[i])))

In [56]:
predictors_coef_sterror = zip(predictors_columns, coef, st_error)

In [57]:
predictors_coef_sterror_matrix = np.array(predictors_coef_sterror)
predictors_coef_sterror_matrix[1]

predictors_coef_sterror_matrix_T = predictors_coef_sterror_matrix.transpose()
predictors = predictors_coef_sterror_matrix_T[0]
coef = predictors_coef_sterror_matrix_T[1].astype(float)
st_error = predictors_coef_sterror_matrix_T[2].astype(float)

print (predictors[:5])
print (coef[:5])
print (st_error[:5])

['Chronic Homeless' 'Veteran Status' 'Days Enrolled' 'Client Age at Entry'
 'Enrollments']
[ -2.72352404e-01   9.61546434e-01   6.03339220e-04   6.50567852e-04
   3.16698099e-04]
[  2.00073824e-16  -1.46384720e-15   3.21539308e-12  -3.44624731e-14
   5.73935300e-14]


In [58]:
import numpy as np
predictors_coef_sterror_matrix_T = predictors_coef_sterror_matrix.transpose()
z_score = np.divide(coef, st_error)
z_score[:5]

array([ -1.36125955e+15,  -6.56862572e+14,   1.87640890e+08,
        -1.88775730e+10,   5.51801046e+09])

In [59]:
statistics = np.column_stack((predictors, coef, st_error, z_score))
statistics[0]

array(['Chronic Homeless', '-0.272352404071', '2.00073823992e-16',
       '-1.36125955228e+15'], 
      dtype='|S88')

In [60]:
# sort predictors by highest to lowest z-score
def getKey(item):
    return item[3]

statistics_sorted = sorted(statistics, key = getKey, reverse = True)

In [61]:
columns = ['predictor', 'coefficient', 'standard error', 'z-score']

output_lr = pd.DataFrame(data = statistics_sorted, columns = columns)
output_lr.head()

,predictor,coefficient,standard error,z-score
0,SCz - Encompass - River Street Shelter,-0.290022444916,-2.91286472913e-16,9.95660533136e+14
1,ZZZ*INACTIVE*ZZZ- MOSBE HRC - SSVF-P2 VRSi- Mo...,-0.468888958935,-4.7761323892e-17,9.81733588448e+15
2,Other TANF-Funded Services,0.52755395346,5.5612262579e-17,9.48628825721e+15
3,SCz-Veterans Resource Center- RRH SSVF -San Ma...,0.124254388382,1.32639959674e-16,9.36779449326e+14
4,zzz*inactive*MOSBE CCHAS (John XXIII) - HOPWA ...,-0.067540632001,-7.23507865989e-18,9.33516208682e+15


# Logistic regression with cross-validation

In [62]:
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
# cross_val_score automatically equals the size of the training and test set
from sklearn.cross_validation import cross_val_score

In [63]:
# create a feature set and target variable
predictors = df_features_numeric.drop(['In Permanent Housing'], axis=1)
targets = df_features_numeric['In Permanent Housing']

In [64]:
## CROSS-VALIDATION DOES NOT IMPROVE THE PREDICTION

# perform 10-fold cross validation
lr_model_crossval = LogisticRegression()

scores = cross_val_score(lr_model_crossval, predictors, targets, cv = 10, scoring = 'accuracy')
print ('scores for all the 10 samples: ')
print (scores)
print ("---")
print ('mean score:')
print (scores.mean())

scores for all the 10 samples: 
[ 0.9         0.95294118  0.94117647  0.95        0.95294118  0.94411765
  0.94411765  0.92330383  0.8879056   0.92035398]
---
mean score:
0.931685753948


# run a logistic regression for each variable seperately

In [65]:
# create a feature set and target variable
predictors_housing_status = df_features_numeric[['Category 1 - Homeless',
                                                 'Category 2 - At imminent risk of losing housing',
                                                'Category 3 - Homeless only under other federal statutes',
                                                'Category 4 - Fleeing domestic violence']]
targets = df_features_numeric['In Permanent Housing']

result.summary()

logit = sm.Logit(targets, predictors_housing_status)
result = logit.fit()

result.pred_table()

# get the marginal effect
marginal_effect_x = result.get_margeff()
print (marginal_effect_x.summary())

NameError: name 'result' is not defined

# (2) Logistic Regression with Statsmodels

In [66]:
import statsmodels.api as sm
from statsmodels.formula.api import logit, probit, poisson, ols
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

In [67]:
import statsmodels.formula.api as smf 

# Make logistic regressions for each variable on 'In Permanent Housing'

In [68]:
# remove spaces in the variables 
df_features = df_features.rename(columns={'In Permanent Housing': 'in_permanent_housing', 
                                          'Non-Cash Benefit': 'non_cash_benefit',
                                          'Chronic Homeless': 'chronic_homeless',
                                          'Domestic Violence Victim': 'domestic_violence_victim',
                                          'Housing Status @ Project Start': 'housing_status_project_start',
                                          'Gender': 'gender',
                                          'Veteran Status': 'veteran_status',
                                          'Days Enrolled': 'days_enrolled',
                                          'Disability Type': 'disability_type',
                                          'Living situation before program entry?': 'living_situation_before_program_entry',
                                          'Race': 'race',
                                          'Client Age at Entry': 'client_age_at_entry',
                                          'Project Name': 'project_name',
                                          'Enrollments': 'enrollments',
                                          'Organization Name': 'organization_name',
                                          'Project Type Code': 'project_type_code',
                                          'Ethnicity': 'ethnicity',
                                          'Continuously Homeless One Year': 'continuously_homeless_one_year'
                                          })
df_features.head(1)

,non_cash_benefit,chronic_homeless,domestic_violence_victim,housing_status_project_start,gender,veteran_status,days_enrolled,disability_type,living_situation_before_program_entry,in_permanent_housing,race,client_age_at_entry,project_name,enrollments,organization_name,project_type_code,ethnicity,continuously_homeless_one_year,Food Stamps,Other Source,Other TANF-Funded Services,"Section 8, Public Housing",TANF Child Care Services,TANF Transportation Services,Temporary rental assistance,WIC,At-risk of homelessness,Category 1 - Homeless,Category 2 - At imminent risk of losing housing,Category 3 - Homeless only under other federal statutes,Category 4 - Fleeing domestic violence,Client doesn't know,Client refused,Data not collected,Stably housed,Female,Male,Other,Transgender female to male,Transgender male to female,Unknown,Alcohol Abuse,Both Alcohol and Drug Abuse,Chronic Health Condition,Developmental,Drug Abuse,HIV/AIDS,Mental Health Problem,None,Physical,Substance Abuse,"Emergency shelter, including hotel or motel paid for with emergency shelter voucher(HUD)",Foster care home or foster care group home,Hospital or other residential non-psychiatric medical facility,Hotel or motel paid for without emergency shelter voucher,"Jail, prison or juvenile detention facility","Owned by client, no ongoing housing subsidy","Owned by client, with ongoing housing subsidy",Permanent housing for formerly homeless persons,Place not meant for habitation,Psychiatric hospital or other psychiatric facility,"Rental by client, no ongoing housing subsidy","Rental by client, with VASH subsidy","Rental by client, with other ongoing housing subsidy",Residential project or halfway house with no homeless criteria,Safe Haven,"Staying or living in a family member's room, apartment or house","Staying or living in a friend's room, apartment or house",Substance abuse treatment facility or detox center,Transitional housing for homeless persons (including homeless youth),American Indian or Alaska Native,Asian,Black or African American,Native Hawaiian or Other Pacific Islander,White,MOSBE SOP - Transitional Housing - Men In Transition,MOSBE - Homeless Winter Shelter for Families,MOSBE CHS - Elm House,MOSBE CHS - RHY - BCP - HP,MOSBE CHS - RHY SOP,MOSBE CHS - Safe Passage,MOSBE CHS - Safe Place Warming Shelter,MOSBE CSUMB - Chinatown Project,MOSBE Franciscan Workers - House of Peace,MOSBE Franciscan Workers - Women Alive! Shelter,MOSBE Franciscan Workers of Junipero Serra,MOSBE HRC - LEGACY Emergency Rental Assistance,MOSBE HRC - SSVF- P1 - CCCIL HP,MOSBE HRC - SSVF- P1 - CCCIL RRH,MOSBE HRC - SSVF- P1 - HRC HP,MOSBE HRC - SSVF- P1 - HRC RRH,MOSBE HRC - SSVF- P1 - VTC HP,MOSBE HRC - SSVF- P1 - VTC RRH,MOSBE HRC - SSVF- P2 HRC- RRH - Santa Cruz Clients,MOSBE HRC - SSVF- P2 - HRC RRH,MOSBE HRC - SSVF- P2 HRC - HP- Monterey/San Benito Clients,MOSBE HRC - SSVF- P2 HRC - RRH- Monterey/San Benito Clients,MOSBE HRC - Security Deposit Guarantees,MOSBE Housing Authority - Pueblo del Mar Family Recovery Community,MOSBE Housing Authority - S + C Vouchers,...,MOSBE SOP - Hamilton ES,MOSBE SOP - Hamilton Family Units -DO NOT USE-,MOSBE SOP - MLP - Emergency Men's Program,MOSBE SOP - Mobile Outreach Shelter Program - MOST,MOSBE SOP - Natividad Shelter,MOSBE SOP - Transitional Housing - Lexington Court,MOSBE SOP - Transitional Housing - Wittenmyer (Homeward Bound),MOSBE SOP - Winter Warming Shelter,MOSBE San Benito County - Helping Hands,MOSBE San Benito County - Homeless Warming Shelter,MOSBE The Salvation Army - Casa de las Palmas,MOSBE The Salvation Army - Fredericksen House,MOSBE The Salvation Army - Good Samaritan Center,MOSBE The Salvation Army - Phase II,MOSBE VTC DO NOT ENTER HERE,MOSBE Veterans Transition Center - GPD Housing,MOSBE Veterans Transition Center - Outreach,SCz - CAB TSP Rental Assistance,SCz - County Office of Education,SCz - Encompass,SCz - Encompass - Anderson,SCz - Encompass - River Street Shelter,SCz - Encompass - River Street Shelter PATH,SCz - FIT - Brom

In [69]:
df_features['non_cash_benefit'].value_counts()

Food Stamps                     2917
WIC                              156
Other Source                     148
Section 8, Public Housing        116
Other TANF-Funded Services        29
TANF Child Care Services          18
TANF Transportation Services      10
Temporary rental assistance        3
Name: non_cash_benefit, dtype: int64

In [70]:
# # logistic regression for Non Cash Benefit

logit_benefit = smf.logit(formula='in_permanent_housing ~ C(non_cash_benefit)', 
                          data=df_features).fit()
# odds ratios
print("Odds Ratios")

print(np.exp(logit_benefit.params))
print(logit_benefit.summary())

         Current function value: 0.313057
         Iterations: 35
Odds Ratios
Intercept                                              9.909570e-02
C(non_cash_benefit)[T.Other Source]                    1.486309e+00
C(non_cash_benefit)[T.Other TANF-Funded Services]      2.632501e+00
C(non_cash_benefit)[T.Section 8, Public Housing]       8.199144e+00
C(non_cash_benefit)[T.TANF Child Care Services]        1.379957e-10
C(non_cash_benefit)[T.TANF Transportation Services]    1.008826e-09
C(non_cash_benefit)[T.Temporary rental assistance]     1.018487e-07
C(non_cash_benefit)[T.WIC]                             4.740858e-01
dtype: float64
                            Logit Regression Results                            
Dep. Variable:     in_permanent_housing   No. Observations:                 3397
Model:                            Logit   Df Residuals:                     3389
Method:                             MLE   Df Model:                            7
Date:                  Wed, 14 Dec 2016

/Users/annalie/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [ ]:
# Logistic Regression for 'Housing Status @ Project Start'

# rename variable
logit_housing_status = smf.logit(formula='in_permanent_housing ~ C(housing_status_project_start)', 
                                 data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_housing_status.params))
print(logit_housing_status.summary())

In [ ]:
# Logistic Regression for 'Living situation before program entry?'

# rename variable
logit_living_situation = smf.logit(formula='in_permanent_housing ~ C(living_situation_before_program_entry)', 
                                   data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_living_situation.params))
print(logit_living_situation.summary())

In [ ]:
## too many iterations, > 35

# Logistic Regression for 'Project Name'

# rename variable
logit_project_name = smf.logit(formula='in_permanent_housing ~ C(project_name)', 
                               data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_project_name.params))
print(logit_project_name.summary())

In [ ]:
# Logistic Regression for 'Project Type Code'

# rename variable
logit_project_type = smf.logit(formula='in_permanent_housing ~ C(project_type_code)', 
                               data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_project_type.params))
print(logit_project_type.summary())

In [ ]:
## too many iterations, > 35

# Logistic Regression for 'Organization Name'

# rename variable
logit_organization_name = smf.logit(formula='in_permanent_housing ~ C(organization_name)', 
                                    data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_organization_name.params))
print(logit_organization_name.summary())

In [ ]:
# Logistic Regression for 'Disability Type'

# rename variable
logit_disability = smf.logit(formula='in_permanent_housing ~ C(disability_type)', 
                             data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_disability.params))
print(logit_disability.summary())

In [ ]:
df_features['race'].value_counts()

In [ ]:
# Logistic Regression for 'Race'

# rename variable
logit_race = smf.logit(formula='in_permanent_housing ~ C(race)', 
                       data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_race.params))
print(logit_race.summary())

In [ ]:
df_features['ethnicity'].value_counts()

In [ ]:
# Logistic Regression for 'Ethnicity'

# rename variable
logit_ethnicity = smf.logit(formula='in_permanent_housing ~ C(ethnicity)', 
                            data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_ethnicity.params))
print(logit_ethnicity.summary())

In [ ]:
df_features['gender'].value_counts()

In [ ]:
# Logistic Regression for 'Gender'

# rename variable
logit_gender = smf.logit(formula='in_permanent_housing ~ C(gender)', 
                            data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_gender.params))
print(logit_gender.summary())

In [ ]:
df_features['chronic_homeless'].value_counts()

In [ ]:
# Logistic Regression for 'Chronic Homeless'

# rename variable
logit_chronic_homeless = smf.logit(formula='in_permanent_housing ~ C(chronic_homeless)', 
                                   data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_chronic_homeless.params))
print(logit_chronic_homeless.summary())

In [ ]:
df_features['domestic_violence_victim'].value_counts()

In [ ]:
# Logistic Regression for 'Domestic Violence Victim'

# rename variable
logit_domestic_violence = smf.logit(formula='in_permanent_housing ~ C(domestic_violence_victim)', 
                                    data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_domestic_violence.params))
print(logit_domestic_violence.summary())

In [ ]:
df_features['veteran_status'].value_counts()

In [ ]:
# Logistic Regression for 'Veteran Status'

# rename variable
logit_veteran = smf.logit(formula='in_permanent_housing ~ C(veteran_status)', 
                          data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_veteran.params))
print(logit_veteran.summary())

In [ ]:
# Logistic Regression for 'Continuously Homeless One Year'

# rename variable
logit_continuously_homeless = smf.logit(formula='in_permanent_housing ~ C(continuously_homeless_one_year)', 
                                        data=df_features).fit()
# odds ratios
print("Odds Ratios")
print(np.exp(logit_continuously_homeless.params))
print(logit_continuously_homeless.summary())

# (2) Logistic Regression with Statsmodels - Matt

In [ ]:
from __future__ import division
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
# import matplotlib as mpl
# import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.metrics import precision_recall_curve, precision_score, recall_score
from sklearn.metrics import f1_score

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import patsy
from IPython.display import display

In [ ]:
# create dummy variables for the following variables
for column in ['Non-Cash Benefit', 
               'Housing Status @ Project Start', 
               'Gender', 
               'Disability Type', 
               'Living situation before program entry?', 
               'Race', 
               'Project Name', 
               'Organization Name', 
               'Project Type Code', 
               'Ethnicity']:
    dummies = pd.get_dummies(df_features[column])
    df_features[dummies.columns] = dummies
    
# create a new dataframe that contains only numeric datatypes
df_features_numeric = df_features.drop(['Non-Cash Benefit', 'Housing Status @ Project Start', 'Domestic Violence Victim', 'Gender', 'Disability Type',
                                        'Living situation before program entry?','Race', 'Project Name', 'Organization Name',
                                        'Project Type Code', 'Ethnicity'], axis=1)

df_features_numeric.head(1)

In [ ]:
# create a feature set and target variable
predictors = df_features_numeric.drop(['In Permanent Housing'], axis=1)
targets = df_features_numeric['In Permanent Housing']

# split data set in training and test set
pred_train, pred_test, tar_train, tar_test = train_test_split(predictors, targets, test_size=.3)

In [ ]:
# run a logistic regression and evaluate model 
classify(pred_train, pred_test, tar_train, tar_test, 'lr')

In [ ]:
# run a logistic regression model for only 'Project Type Code

# create dataframe for only 'Project Type Code' with dummy variables
df_features_project_type = df_features[['In Permanent Housing', 'Project Type Code']]

# create dummy variables for the following variables
dummies_project_type = pd.get_dummies(df_features_project_type['Project Type Code'])
df_features_project_type[dummies_project_type.columns] = dummies_project_type

# create a new dataframe that contains only numeric datatypes
df_features_numeric_project_type = df_features_project_type.drop('Project Type Code', axis=1)

In [ ]:
# create a feature set and target variable for 'Project Type Code' dataframe
predictors_project_type = df_features_numeric_project_type.drop(['In Permanent Housing'], axis=1)
# targets = df_features_numeric['In Permanent Housing']

# split data set in training and test set
pt_pred_train, pt_pred_test, pt_tar_train, pt_tar_test = train_test_split(predictors_project_type, targets, test_size=.3)

In [ ]:
# run a logistic regression for 'Project Type Code' dataframe and evaluate model 
classify(pt_pred_train, pt_pred_test, pt_tar_train, pt_tar_test, 'lr')

In [ ]:
# create df_pos and df_neg 

# meaning: a dataframe that contains the succesful cases = when 'In Permanent Housing' = 1  
# and a dataframe that contains the unsuccessful cases = when 'In Permanent Housing' = 0

df_pos = df_features_numeric[df_features_numeric['In Permanent Housing'] == 1]
df_neg = df_features_numeric[df_features_numeric['In Permanent Housing'] == 0]

plot_hist(df_pos, df_neg, pos_str='Positive class', neg_str='Negative class')

In [ ]:
# create df_pos and df_neg for 'Project Type Code' dataframe

# meaning: a dataframe that contains the succesful cases = when 'In Permanent Housing' = 1  
# and a dataframe that contains the unsuccessful cases = when 'In Permanent Housing' = 0

pt_df_pos = df_features_numeric_project_type[df_features_numeric_project_type['In Permanent Housing'] == 1]
pt_df_neg = df_features_numeric_project_type[df_features_numeric_project_type['In Permanent Housing'] == 0]

plot_hist(pt_df_pos, pt_df_neg, pos_str='Positive class', neg_str='Negative class')

In [ ]:
def chunkify(lst, n=1, to_remove=None):
    """Chunk a list into n approximately equal groups. Also remove hardcoded items."""
    if to_remove is not None:
        lst = [x for x in lst if x not in to_remove]
    return [ lst[i::n] for i in xrange(n) ]

In [ ]:
def sm_logit(df, f=None, features=None,
             outcome='outcome_field',
             add_constant=True,
             categorical=None,
             maxiter=35,
             reg_method=None,
             reg_alpha=1.0,
             #log_trans=None,
             #sort_by='z',
             #print_action=None,
             ):
    """reg_method can be 'l1' or 'l1_cvxopt_cp'
    """
    if features is not None:
        df = df[features]
    else:
        features = df.columns.tolist()

    if add_constant:
        df = sm.tools.add_constant(df, prepend=False, has_constant='raise')

    if f is None:
        these_features = [x for x in features if x != outcome]
        if categorical is not None:
            f = '{} ~ '.format(outcome) + ' + '.join(['C({})'.format(x) if x in categorical else x for x in these_features])
        else:
            f = '{} ~ '.format(outcome) + ' + '.join([x for x in these_features])

    if reg_method is not None:
        y, X = patsy.dmatrices(f, df, return_type='dataframe')

        #if reg_method == 'l1':
        #    reg_alpha = 1.0 # pure lasso
        #elif reg_method == 'l2':
        #    reg_alpha = # ridge

        # higher alpha = more coeff equal to zero
        reg_alpha = reg_alpha * np.ones(X.shape[1])
        reg_alpha[X.columns.tolist().index('Intercept')] = 0

        results_log = sm.Logit(y, X, missing='drop').fit_regularized(method=reg_method, alpha=reg_alpha)
    else:
        results_log = sm.Logit.from_formula(f, df, missing='raise').fit(maxiter=maxiter)

    #print_sm_logit_results(results_log, sort_by=sort_by, log_trans=log_trans, print_action=print_action)

    return results_log

In [ ]:
def print_sm_logit_results(results, sort_by='z', log_trans=None, print_n=None, print_p_limit=.05, print_action=None):
    #summary = results.summary()
    summary = results.summary2()
    display(summary.tables[0])

    if print_action is None:
        print_action = ''
    if len(print_action) > 0 and print_action[0] != ' ':
        print_action = ' {}'.format(print_action)

    #if log_trans is not None:
    #    if isinstance(log_trans, str):
    #        log_trans = [log_trans]
    #    for feat in log_trans:
    #        if feat in results.params.index.tolist():
    #            results.params.ix[feat] = np.exp(results.params.ix[feat])

    coef_str = 'Coef.'
    odds_str = 'Odds Ratio'
    p_str = 'P>|z|'
    results_print = pd.DataFrame(np.array([results.params, np.exp(results.params)]).T, index=results.params.index, columns=[coef_str, odds_str])
    results_print.loc[results_print[odds_str] < 1, odds_str] = results_print.loc[results_print[odds_str] < 1, odds_str].apply(lambda x: 1/x)

    ci = results.conf_int()
    ci.columns = ['[0.025', '0.975]']

    results_print = results_print.join(
        pd.DataFrame(results.bse, columns=['Std.Err.'])).join(
        pd.DataFrame(results.tvalues, columns=['z'])).join(
        pd.DataFrame(results.pvalues, columns=[p_str])).join(
        ci)
    results_more = results_print[(results_print[coef_str] >= 0) & (results_print[p_str] <= print_p_limit)].sort_values(by=sort_by, ascending=False)
    results_less = results_print[(results_print[coef_str] < 0) & (results_print[p_str] <= print_p_limit)].sort_values(by=sort_by, ascending=True)

    if print_n is None:
        print_n = results_more.shape[0]

    print('More likely{}:'.format(print_action))
    if print_n > results_more.shape[0]:
        print_n = results_more.shape[0]
    display(results_more.iloc[:print_n, :])

    print('\nLess likely{}:'.format(print_action))
    if print_n > results_less.shape[0]:
        print_n = results_less.shape[0]
    display(results_less.iloc[:print_n, :])

    if len(summary.tables) > 2:
        for i in range(2, len(summary.tables)):
            display(summary.tables[i])
    return (results_more, results_less)

In [ ]:
def df_find_existing_col(df, col, result_col=None):
    '''Find and return the columns in list col that exist in dataframe df,
    returning that intersecting subset (and corresponding entries of result_col).
    '''
    if result_col == None:
        col = list(set(df.columns.tolist()).intersection(col))
    else:
        if len(col) != len(result_col):
            logging.warning('col and result_col are not the same length.')
            return
        # make a dictionary out of the columns we want in the numerator
        ind_dict = dict((k,i) for i,k in enumerate(col))
        # find the overlap between requested columns and what actually exists in the df
        inter = set(ind_dict).intersection(df.columns.tolist())
        # get their indices and overwrite with overlapping set
        indices = [ind_dict[x] for x in inter]
        col = [col[i] for i in indices]
        result_col = [result_col[i] for i in indices]
    if len(col) == 0:
        logging.warning('None of the columns are in the dataframe.')
    return (col, result_col)

In [ ]:
def quantile_nonzero_values(df, regex_col='^namestart', quant_dim='quantity', q=4, quant_col_prefix='', col_prefix_to_remove=None):
    """Calculate quantile of quant_dim for regex_col regex match with >0 values
    """
    cols = df.filter(regex=regex_col).columns.tolist()
    if len(cols) > 0:
        for col in cols:
            if col_prefix_to_remove is not None:
                col_rm = col.replace('prefix_', '')
            else:
                col_rm = col
            quant_col = '{qcp}{cr}_quant'.format(qcp=quant_col_prefix,cr=col_rm)
            # fill with nan so we can operate on non-zero values later
            df.loc[:, quant_col] = np.nan
            if len(df[(df[col] > 0)][quant_dim]) > 0 and df[(df[col] > 0)][quant_dim].std() > 0:
                labels = range(1, q+1)
                # # option 1: use qcut
                # col_q = pd.qcut(df[(df[col] > 0)][quant_dim], q, labels=range(1, q+1))

                # # option 2: manual bins for when there might not be unique bins, only based on unique values
                # bins = pd.core.algorithms.quantile(np.unique(df[(df[col] > 0)][quant_dim]),
                #                                    np.linspace(0, 1, q+1))

                # option 3: manual bins for when there might not be unique bins, only based on unique values
                bins = pd.core.algorithms.quantile(df[(df[col] > 0)][quant_dim],
                                                   np.linspace(0, 1, q+1))
                if len(np.unique(bins)) < len(labels):
                    bins = np.unique(bins)
                    # # option 1: change q to length of unique bins
                    # labels = range(1, len(bins))

                    # option 2: interpolate bin numbers, keeping min and max, skipping where needed
                    labels = np.round(np.linspace(1, len(labels), len(bins)-1))

                col_q = pd.tools.tile._bins_to_cuts(df[(df[col] > 0)][quant_dim],
                                                    bins, labels=labels, include_lowest=True)
                df.loc[(df[col] > 0), quant_col] = col_q
            #df[quant_col] = df[quant_col].fillna(np.nan)
    return df

In [ ]:
def calc_similarity(df, columns=None, field_prefix=''):
    '''Calculates similarity, using two different measures: Schmidt number and entropy.
    The Schmidt number (K) measures the factorability of a matrix in terms of the singular values from SVD.
    Here, 1-K it represents how broadly the rows (independent events) sample the possible space.
    If every row is the same (every event is the same every time), K=1. K increases with variability across rows.
    The maximum value of K is either N columns (components that make up an event) or M rows (events), whichever is smaller.
    This maximum value is achieved when rows contain the same frequency of the different columns.
    NB: This function returns 1 minus a normalized K, normalized by its maximum possible value
        to keep it within the interval [0, 1]. It is subtracted from 1 to give a similarity value,
        where 0 means completely dissimilar and 1 means completely similar.
    For the entropy measure, it returns e^(-entropy), to give a measure of similarity.
    This will return 1 when events are identical and moves toward 0 as they differ.
    '''
    if columns is None:
        columns = df.columns.tolist()
    freq = df.as_matrix(columns=columns).T
    U, s, Vh = svd(freq)
    s_norm = s / s.sum()

    # schmidt decomposition-based similarity
    K = 1 / (s_norm**2).sum()
    # normalize the schmidt number by the maximum possible value of K: (K - 1) / [ min(M,N) - 1]
    K_norm = (K - 1) / (min(freq.shape) - 1)

    # entropy-based similarity
    entropy = np.nansum(-np.log(s_norm) * s_norm)

    return pd.Series({'{}similarity_schmidt'.format(field_prefix): 1 - K_norm,
                      '{}similarity_entropy'.format(field_prefix): np.exp(-entropy)})

In [ ]:
def classify(X_train, X_test, y_train, y_test, classifier,
             class_weight=None,
             penalty='l1', n_estimators=200,
             min_samples_split=2, min_samples_leaf=1,
             n_jobs=10, verbose=0, print_n_features=50,
             print_it=True, plot_it=True):
    # true class is second column (y_pred_proba[:,1])
    if (classifier == 'lr'):
        myLR = LogisticRegression(penalty=penalty, class_weight=class_weight, n_jobs=n_jobs, verbose=verbose)
        myLR.fit(X_train, y_train)
        y_pred = myLR.predict(X_test)
        y_pred_proba = myLR.predict_proba(X_test)
    elif (classifier == 'rf'):
        myRF = RandomForestClassifier(n_estimators=n_estimators, class_weight=class_weight, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, n_jobs=n_jobs, verbose=verbose)
        myRF.fit(X_train, y_train)
        y_pred = myRF.predict(X_test)
        y_pred_proba = myRF.predict_proba(X_test)

    if len(np.unique(y_train)) == 2 & len(np.unique(y_test)) == 2:
        fpr, tpr, roc_thresholds = roc_curve(y_test, y_pred_proba[:,1])
        myAuc = auc(fpr,tpr)
    else:
        myAuc = None

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = 2*((precision * recall) / (precision + recall))

    if print_it:
        if len(np.unique(y_train)) == 2 & len(np.unique(y_test)) == 2:
            print('\nAUC: %.5f' % myAuc)
        print('Precision: %.4f' % precision)
        print('Recall: %.4f' % recall)
        print('F1 score: %.4f' % f1)

        print('\nConfusion Matrix')
        #print(confusion_matrix(y_test, y_pred))
        print(pd.crosstab(y_test, y_pred, rownames=['Truth'], colnames=['Prediction'], margins=True))
        print('\nPercent of outcomes classified')
        print(pd.crosstab(y_test, y_pred, rownames=['Truth'], colnames=['Prediction']).apply(lambda r: 100.0 * r/r.sum()))

    if (classifier == 'lr'):
        myModel = myLR
        results = pd.DataFrame(np.array([myLR.coef_[0], np.exp(myLR.coef_[0])]).T, index=X_train.columns, columns=['coef', 'odds'])
        results = results.reindex(results['coef'].abs().sort_values(ascending=False).index)
        results.loc[results['odds'] < 1, 'odds'] = results.loc[results['odds'] < 1, 'odds'].apply(lambda x: 1/x)
        if print_it:
            print('\nAs feature increases, more likely to be in positive class:')
            print(results[results['coef'] >= 0].head(print_n_features))
            print('\nAs feature increases, less likely be in positive class:')
            print(results[results['coef'] < 0].head(print_n_features))
    elif (classifier == 'rf'):
        myModel = myRF
        results = pd.DataFrame(myRF.feature_importances_, index=X_train.columns, columns=['importance'])
        results = results.sort_values(['importance'], ascending=False)
        if print_it:
            print('\nFeatures, ranked by importance')
            print(results.head(print_n_features))

    if plot_it & (len(np.unique(y_train)) == 2 & len(np.unique(y_test)) == 2):
        precision_curve, recall_curve, pr_thresholds = precision_recall_curve(y_test, y_pred_proba[:,1])
        f1_curve = 2*((precision_curve * recall_curve) / (precision_curve + recall_curve))

        ncols = 3
        nrows = 1
        fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols*5,nrows*5))

        ax[0].plot(fpr, tpr)
        ax[0].set_aspect('equal')
        ax[0].set_xlabel('False Positive Rate')
        ax[0].set_ylabel('True Positive Rate')
        ax[0].set_title('ROC [AUC=%0.3f]' % (myAuc))

        ax[1].plot(roc_thresholds, tpr, 'b')
        ax[1].plot(roc_thresholds, fpr, 'r')
        ax[1].set_xlim((0, 1))
        ax[1].set_aspect('equal')
        ax[1].legend(['True Pos Rate', 'False Pos Rate'])
        ax[1].set_xlabel('Threshold')

        ax[2].plot(pr_thresholds, precision_curve[:-1], 'b')
        ax[2].plot(pr_thresholds, recall_curve[:-1], 'r')
        ax[2].plot(pr_thresholds, f1_curve[:-1], 'g')
        ax[2].set_xlim((0, 1))
        ax[2].set_aspect('equal')
        ax[2].legend(['Precision', 'Recall', 'F1 Score'])
        ax[2].set_xlabel('Threshold')
    return (results, myAuc, precision, recall, f1, myModel)

def plot_hist(df_pos, df_neg, pos_str='Positive class', neg_str='Negative class',
              results=None, n=None,
              max_col=4, normed=False,
              sharex=False, sharey=False,
              figsize_multuplier=5):
    dummy_str = 'null999'
    if results is None:
        these_cols = df_pos.columns.tolist()
    elif type(results) is list:
        these_cols = results
    elif type(results) is str:
        these_cols = [results]
    elif hasattr(results, 'index'):
        if n is None:
            n=40
        these_cols = results.index.tolist()[:n]

    if max_col < 2:
        max_col = 2
    if len(these_cols) == 1:
        these_cols.append(dummy_str)
    if len(these_cols) < max_col:
        ncols = len(these_cols)
    else:
        ncols = max_col
    nrows = int(np.ceil(len(these_cols)/ncols))

    fig, axes = plt.subplots(ncols=ncols, nrows=nrows,
                             figsize=(ncols*figsize_multuplier,nrows*figsize_multuplier),
                             sharex=sharex, sharey=sharey)

    count = -1
    for ax, cat in zip(axes.ravel(),these_cols):
        if cat == dummy_str:
            continue
        count += 1
        min_val = min(int(df_pos[cat].min()), df_neg[cat].min())
        max_val = max(int(df_pos[cat].max()), int(df_neg[cat].max()))
        if max_val <= 1:
            bin_spacing = .1
            if min_val == 0:
                bin_spacing = .25
                if max_val == 1:
                    max_val += .1
                elif max_val == 0:
                    max_val = 1.1
        elif max_val < 4:
            bin_spacing = .2
        elif max_val < 50:
            bin_spacing = .5
        elif max_val < 100:
            bin_spacing = 1
        elif max_val < 500:
            bin_spacing = 5
        elif max_val < 1000:
            bin_spacing = 10
        elif max_val < 5000:
            bin_spacing = 50
        elif max_val < 10000:
            bin_spacing = 100
        elif max_val < 50000:
            bin_spacing = 500
        else:
            bin_spacing = 1000

        bins = np.arange(min_val, max_val, bin_spacing)
        try:
            df_pos[cat].plot(ax=ax, kind='hist', alpha=.5, color='red', normed=normed, bins=bins);
        except:
            print('pos {}'.format(cat))
            print('min_val {}'.format(min_val))
            print('max_val {}'.format(max_val))
            print('bin_spacing {}'.format(bin_spacing))
            print(bins)
        try:
            df_neg[cat].plot(ax=ax, kind='hist', alpha=.5, color='blue', normed=normed, bins=bins);
        except:
            print('neg {}'.format(cat))
            print('min_val {}'.format(min_val))
            print('max_val {}'.format(max_val))
            print('bin_spacing {}'.format(bin_spacing))
            print(bins)
        if hasattr(results, 'columns'):
            if 'coef' in results.columns:
                title_str = cat
                if results.ix[count, 'coef'] < 0:
                    title_str = '%s: %.1f' % (title_str, -results.ix[count, 'odds'])
                elif results.ix[count, 'coef'] > 0:
                    title_str = '%s: %.1f' % (title_str, results.ix[count, 'odds'])
            elif 'importance' in results.columns:
                #title_str = '%s: %.2f' % (cat, results.ix[count, 'importance'])
                title_str = cat
            else:
                title_str = cat
        else:
            title_str = cat
        ax.set_title(title_str)
        ax.set_ylabel('Count')
        ax.legend([pos_str, neg_str], loc='upper right');
    fig.tight_layout()

def chunkify(lst, n=1, to_remove=None):
    """Chunk a list into n approximately equal groups. Also remove hardcoded items."""
    if to_remove is not None:
        lst = [x for x in lst if x not in to_remove]
    return [ lst[i::n] for i in xrange(n) ]

def sm_logit(df, f=None, features=None,
             outcome='outcome_field',
             add_constant=True,
             categorical=None,
             maxiter=35,
             reg_method=None,
             reg_alpha=1.0,
             #log_trans=None,
             #sort_by='z',
             #print_action=None,
             ):
    """reg_method can be 'l1' or 'l1_cvxopt_cp'
    """
    if features is not None:
        df = df[features]
    else:
        features = df.columns.tolist()

    if add_constant:
        df = sm.tools.add_constant(df, prepend=False, has_constant='raise')

    if f is None:
        these_features = [x for x in features if x != outcome]
        if categorical is not None:
            f = '{} ~ '.format(outcome) + ' + '.join(['C({})'.format(x) if x in categorical else x for x in these_features])
        else:
            f = '{} ~ '.format(outcome) + ' + '.join([x for x in these_features])

    if reg_method is not None:
        y, X = patsy.dmatrices(f, df, return_type='dataframe')

        #if reg_method == 'l1':
        #    reg_alpha = 1.0 # pure lasso
        #elif reg_method == 'l2':
        #    reg_alpha = # ridge

        # higher alpha = more coeff equal to zero
        reg_alpha = reg_alpha * np.ones(X.shape[1])
        reg_alpha[X.columns.tolist().index('Intercept')] = 0

        results_log = sm.Logit(y, X, missing='drop').fit_regularized(method=reg_method, alpha=reg_alpha)
    else:
        results_log = sm.Logit.from_formula(f, df, missing='raise').fit(maxiter=maxiter)

    #print_sm_logit_results(results_log, sort_by=sort_by, log_trans=log_trans, print_action=print_action)

    return results_log

def print_sm_logit_results(results, sort_by='z', log_trans=None, print_n=None, print_p_limit=.05, print_action=None):
    #summary = results.summary()
    summary = results.summary2()
    display(summary.tables[0])

    if print_action is None:
        print_action = ''
    if len(print_action) > 0 and print_action[0] != ' ':
        print_action = ' {}'.format(print_action)

    #if log_trans is not None:
    #    if isinstance(log_trans, str):
    #        log_trans = [log_trans]
    #    for feat in log_trans:
    #        if feat in results.params.index.tolist():
    #            results.params.ix[feat] = np.exp(results.params.ix[feat])

    coef_str = 'Coef.'
    odds_str = 'Odds Ratio'
    p_str = 'P>|z|'
    results_print = pd.DataFrame(np.array([results.params, np.exp(results.params)]).T, index=results.params.index, columns=[coef_str, odds_str])
    results_print.loc[results_print[odds_str] < 1, odds_str] = results_print.loc[results_print[odds_str] < 1, odds_str].apply(lambda x: 1/x)

    ci = results.conf_int()
    ci.columns = ['[0.025', '0.975]']

    results_print = results_print.join(
        pd.DataFrame(results.bse, columns=['Std.Err.'])).join(
        pd.DataFrame(results.tvalues, columns=['z'])).join(
        pd.DataFrame(results.pvalues, columns=[p_str])).join(
        ci)
    results_more = results_print[(results_print[coef_str] >= 0) & (results_print[p_str] <= print_p_limit)].sort_values(by=sort_by, ascending=False)
    results_less = results_print[(results_print[coef_str] < 0) & (results_print[p_str] <= print_p_limit)].sort_values(by=sort_by, ascending=True)

    if print_n is None:
        print_n = results_more.shape[0]

    print('More likely{}:'.format(print_action))
    if print_n > results_more.shape[0]:
        print_n = results_more.shape[0]
    display(results_more.iloc[:print_n, :])

    print('\nLess likely{}:'.format(print_action))
    if print_n > results_less.shape[0]:
        print_n = results_less.shape[0]
    display(results_less.iloc[:print_n, :])

    if len(summary.tables) > 2:
        for i in range(2, len(summary.tables)):
            display(summary.tables[i])
    return (results_more, results_less)

def df_find_existing_col(df, col, result_col=None):
    '''Find and return the columns in list col that exist in dataframe df,
    returning that intersecting subset (and corresponding entries of result_col).
    '''
    if result_col == None:
        col = list(set(df.columns.tolist()).intersection(col))
    else:
        if len(col) != len(result_col):
            logging.warning('col and result_col are not the same length.')
            return
        # make a dictionary out of the columns we want in the numerator
        ind_dict = dict((k,i) for i,k in enumerate(col))
        # find the overlap between requested columns and what actually exists in the df
        inter = set(ind_dict).intersection(df.columns.tolist())
        # get their indices and overwrite with overlapping set
        indices = [ind_dict[x] for x in inter]
        col = [col[i] for i in indices]
        result_col = [result_col[i] for i in indices]
    if len(col) == 0:
        logging.warning('None of the columns are in the dataframe.')
    return (col, result_col)

def quantile_nonzero_values(df, regex_col='^namestart', quant_dim='quantity', q=4, quant_col_prefix='', col_prefix_to_remove=None):
    """Calculate quantile of quant_dim for regex_col regex match with >0 values
    """
    cols = df.filter(regex=regex_col).columns.tolist()
    if len(cols) > 0:
        for col in cols:
            if col_prefix_to_remove is not None:
                col_rm = col.replace('prefix_', '')
            else:
                col_rm = col
            quant_col = '{qcp}{cr}_quant'.format(qcp=quant_col_prefix,cr=col_rm)
            # fill with nan so we can operate on non-zero values later
            df.loc[:, quant_col] = np.nan
            if len(df[(df[col] > 0)][quant_dim]) > 0 and df[(df[col] > 0)][quant_dim].std() > 0:
                labels = range(1, q+1)
                # # option 1: use qcut
                # col_q = pd.qcut(df[(df[col] > 0)][quant_dim], q, labels=range(1, q+1))

                # # option 2: manual bins for when there might not be unique bins, only based on unique values
                # bins = pd.core.algorithms.quantile(np.unique(df[(df[col] > 0)][quant_dim]),
                #                                    np.linspace(0, 1, q+1))

                # option 3: manual bins for when there might not be unique bins, only based on unique values
                bins = pd.core.algorithms.quantile(df[(df[col] > 0)][quant_dim],
                                                   np.linspace(0, 1, q+1))
                if len(np.unique(bins)) < len(labels):
                    bins = np.unique(bins)
                    # # option 1: change q to length of unique bins
                    # labels = range(1, len(bins))

                    # option 2: interpolate bin numbers, keeping min and max, skipping where needed
                    labels = np.round(np.linspace(1, len(labels), len(bins)-1))

                col_q = pd.tools.tile._bins_to_cuts(df[(df[col] > 0)][quant_dim],
                                                    bins, labels=labels, include_lowest=True)
                df.loc[(df[col] > 0), quant_col] = col_q
            #df[quant_col] = df[quant_col].fillna(np.nan)
    return df

def calc_similarity(df, columns=None, field_prefix=''):
    '''Calculates similarity, using two different measures: Schmidt number and entropy.
    The Schmidt number (K) measures the factorability of a matrix in terms of the singular values from SVD.
    Here, 1-K it represents how broadly the rows (independent events) sample the possible space.
    If every row is the same (every event is the same every time), K=1. K increases with variability across rows.
    The maximum value of K is either N columns (components that make up an event) or M rows (events), whichever is smaller.
    This maximum value is achieved when rows contain the same frequency of the different columns.
    NB: This function returns 1 minus a normalized K, normalized by its maximum possible value
        to keep it within the interval [0, 1]. It is subtracted from 1 to give a similarity value,
        where 0 means completely dissimilar and 1 means completely similar.
    For the entropy measure, it returns e^(-entropy), to give a measure of similarity.
    This will return 1 when events are identical and moves toward 0 as they differ.
    '''
    if columns is None:
        columns = df.columns.tolist()
    freq = df.as_matrix(columns=columns).T
    U, s, Vh = svd(freq)
    s_norm = s / s.sum()

    # schmidt decomposition-based similarity
    K = 1 / (s_norm**2).sum()
    # normalize the schmidt number by the maximum possible value of K: (K - 1) / [ min(M,N) - 1]
    K_norm = (K - 1) / (min(freq.shape) - 1)

    # entropy-based similarity
    entropy = np.nansum(-np.log(s_norm) * s_norm)

    return pd.Series({'{}similarity_schmidt'.format(field_prefix): 1 - K_norm,
                      '{}similarity_entropy'.format(field_prefix): np.exp(-entropy)})

# Further analysis

- compare people that have a positive outcome or negative outcome with each other, e.g. by plots